In [ ]:
#@title Drive mounting
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title tar file

!tar -zxvf /2022_AICP_edges+nodes_compilations.tar.gz -C /AICP2022/ 2> /dev/null

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
#우리가 만들 것
mol_id = np.array([], dtype='<U10')
num_atoms = np.array([], dtype=int)
num_bonds = np.array([], dtype=int)
x = []
one_hot = []
atomic_numbers = []
edge = []
mu = np.array([])
smiles = np.array([])

In [ ]:
atoms = {'H':1, 'C':6, 'O':8, 'N':7, 'S':16, 'P':15, 'F':9}

In [ ]:
one_hots = {'H':[True, False, False, False, False, False, False], 
            'C':[False, True, False, False, False, False, False], 
            'O':[False, False, True, False, False, False, False], 
            'N':[False, False, False, True, False, False, False], 
            'S':[False, False, False, False, True, False, False],
            'P':[False, False, False, False, False, True, False],
            'F':[False, False, False, False, False, False, True]}

In [ ]:
file_pull_1 = pd.read_csv('/max_pulling_force.csv',
                        header=None)
file_pull_2 = pd.read_csv('/max_pulling_force_1000-1999.csv',
                        header=None)
file_pull_3 = pd.read_csv('/max_pulling_force_2000-2999.csv',
                        header=None)

In [ ]:
file_pull = pd.concat([file_pull_1, file_pull_2, file_pull_3])

In [ ]:
file_pull['id'] = file_pull.iloc[:,0].apply(lambda x:x[:-1])
file_pull['force'] = file_pull.iloc[:,1].apply(lambda x:float(x))

In [ ]:
file_pull['force_norm'] = (file_pull['force']-file_pull['force'].mean())/file_pull['force'].std()

In [ ]:
check = file_pull['id'].to_list()

In [ ]:
file_pull['force_norm'].max()

4.544946871645891

In [ ]:
file_pull[file_pull['force_norm']==file_pull['force_norm'].max()]

,0,1,id,force,force_norm
152,UNK_5A83B4\n,776.404,UNK_5A83B4,776.404,4.544947


H = 1
C = 6
O = 8 
N = 7
P = 15
S = 16
F = 9

In [ ]:
list(range(3000,3001))

[3000]

In [ ]:
errors = 0
for i in range(3000,3001):
  
  # get into file
  path = '/2022_AICP_edges+nodes_compilations/'+str(i)

  # mol id 
  file_id = glob.glob(path+'/*.gro')
  a = file_id[0].split('/')[-1][:-4]

  # checking error
  if a not in check:
    print(a,' has error')
    errors += 1
    pass

  else:
    mol_id = np.append(mol_id, a)

    # atom info -> num_atoms atomic_numbes, x, one_hot
    file_atom = pd.read_csv(path+'/nodes.csv', index_col=0 )
    num_atoms = np.append(num_atoms, int(file_atom.shape[0]))
    file_atom['atom_number']=file_atom['atom_name'].apply(lambda x:atoms[x[0]])

    bb = file_atom['atom_number'].to_numpy(dtype=int)
    b = np.append(bb, np.array((64-len(bb))*[0])).reshape(-1,1)
    atomic_numbers.append(b)

    cc = file_atom[['x_coord','y_coord','z_coord']].to_numpy()
    c = np.append(cc,np.zeros(shape=(64-len(bb),3)), axis=0)
    x.append(c)

    file_atom['one_hot'] = file_atom['atom_name'].apply(lambda x:one_hots[x[0]])
    jj = file_atom['one_hot'].tolist()
    jj = np.array(jj)
    j = np.append(jj, np.array((64-len(bb))*[True,True,True,True,True,True,True]).reshape(-1,7), axis=0)
    one_hot.append(j)

    # edge info -> num_bonds, edge
    file_edge = pd.read_csv(path+'/edges.csv', index_col=0, header=2)[:-1]
    num_bonds = np.append(num_bonds, int(file_edge.shape[0]))
    file_edge['edges_1'] = file_edge.iloc[:,1].apply(lambda x:int(x.split()[0])-1)
    file_edge['edges_2'] = file_edge.iloc[:,1].apply(lambda x:int(x.split()[1])-1)
    file_edge['edge'] = 0
    dd = file_edge[['edges_1', 'edges_2', 'edge']].to_numpy(dtype=int)
    d = np.append(dd,np.zeros(shape=(67-len(dd),3)), axis=0)
    edge.append(d)

    # pulling force
    e = file_pull[file_pull['id']==a]['force_norm']
    mu = np.append(mu, e)

    #smiles
    file_smiles = open(path+'/smiles.txt','r')
    f = file_smiles.read()
    smiles = np.append(smiles, f)
  print(i)

UNK_2D0248  has error
3000


In [ ]:
max(num_atoms)

64

In [ ]:
# max atom
print('max numebr of atom: ',max(num_atoms))
# max edge
print('max number of bond: ',max(num_bonds))

max numebr of atom:  64
max number of bond:  67


In [ ]:
errors

1

In [ ]:
x = np.array(x)
atomic_numbers = np.array(atomic_numbers,dtype=int)
edge = np.array(edge,dtype=int)
one_hot = np.array(one_hot, dtype=bool)

In [ ]:
# pt file
data = {}

data['mol_id']=mol_id
data['num_atoms']=num_atoms
data['num_bonds']=num_bonds
data['x']=x
data['one_hot']=one_hot
data['atomic_numbers']=atomic_numbers
data['edge']=edge
data['mu']=mu
data['smiles']=smiles

In [ ]:
data['mu'].shape

(2872,)

In [ ]:
import torch

In [ ]:
torch.save(data, './amino_train_data.pt')

In [ ]:
data

{'mol_id': array(['UNK_38442F', 'UNK_EF0C2A', 'UNK_881BE7', ..., 'UNK_F666F8',
        'UNK_E410F1', 'UNK_8CF27F'], dtype='<U10'),
 'num_atoms': array([33, 34, 36, ..., 45, 46, 48]),
 'num_bonds': array([32, 33, 35, ..., 44, 45, 47]),
 'x': array([[[ 1.   ,  1.   ,  0.   ],
         [-0.522,  1.   ,  0.   ],
         [-0.903,  1.   ,  1.028],
         ...,
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ]],
 
        [[ 1.   ,  1.   ,  0.   ],
         [-0.522,  1.   ,  0.   ],
         [-0.903,  1.   ,  1.027],
         ...,
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ]],
 
        [[ 1.   ,  1.   ,  0.   ],
         [-0.525,  1.   ,  0.   ],
         [-0.884,  1.   ,  1.036],
         ...,
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ],
         [ 0.   ,  0.   ,  0.   ]],
 
        ...,
 
        [[ 1.   ,  1.   ,  0.   ],
         [-0.521,  1.   ,  0.